In [ ]:
!pip install -q trino

In [ ]:
CATALOG_URL = "http://server:8181/catalog"
TRINO_URI = "https://trino-proxy:443"
KEYCLOAK_TOKEN_ENDPOINT = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"
WAREHOUSE = "demo"

CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"

# Connect to Trino
In this example trino is configured to also use oauth. We are now logging into trino as a human user `peter`.
The following cell should print a link. Open this link in a new tab and login as:
* User: `peter`
* Password: `iceberg`

In [ ]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication
from redirect_handler import REDIRECT_HANDLER
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
)

In [ ]:
cur = conn.cursor()
print("\nResult of 'SELECT 1: ", cur.execute("Select 1").fetchall())
print("Result of 'SELECT current_user': ", cur.execute("SELECT current_user").fetchall())
# The current_user has now the unique ID from our IdP!

# Creating the trino Catalog
Make sure to follow the instructions in the `01-Bootstrap.ipynb` and `02-Create-Warehouse.ipynb` notebooks first!

In [ ]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'OAUTH2',
        "iceberg.rest-catalog.oauth2.credential" = '{CLIENT_ID}:{CLIENT_SECRET}',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "iceberg.rest-catalog.oauth2.scope" = 'lakekeeper',
        "iceberg.rest-catalog.oauth2.server-uri" = '{KEYCLOAK_TOKEN_ENDPOINT}',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""
)

In [ ]:
cur.execute("CREATE SCHEMA IF NOT EXISTS lakekeeper.trino_namespace")
print(cur.execute("SHOW SCHEMAS FROM lakekeeper").fetchall())

# Use the Catalog (User 1: Peter)
Now that the catalog is created, we can use it as usual. Lets login again (if asked) as `peter`, this time directly to the catalog so that we don't have to prefix schemas with `lakekeeper`:

In [ ]:
conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
    catalog="lakekeeper" # This line is new
)

In [ ]:
cur = conn.cursor()
cur.execute("CREATE SCHEMA IF NOT EXISTS trino_namespace")

In [ ]:
cur.execute(
    "CREATE TABLE IF NOT EXISTS trino_namespace.my_table (my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')"
)
cur.execute("INSERT INTO trino_namespace.my_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')")
print(cur.execute("SELECT * FROM trino_namespace.my_table").fetchall())

# Use the Catalog (User 2: Anna)
So far we have just used one user to query our data. We wouldn't need OPA for this.
Lets setup a second user: Anna.

1. Open a new private browser window, open `http://localhost:8181` (Lakekeeper UI) and login using Username `anna` and Password `iceberg`. Anna has no permissions yet. That's alright.
2. Execute the following cell - copy the shown login link into the private browser you used before and re-login as `anna` if asked. The cell execution should fail, as Anna has no permissions yet.
3. In your regular browser, navigate to the Warehouse "demo" and grant anna the "select" permission. Now re-run the cell below. Copy the login link to your private browser tab again.

In [ ]:
conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
    catalog="lakekeeper" # This line is new
)

cur = conn.cursor()
print(cur.execute("SELECT * FROM trino_namespace.my_table").fetchall())

As you can see, trino enforces lakekeeper permissions via OPA!